In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

# Constants
a_max = 1.0  # Maximum acceleration magnitude allowed

# Dynamics of the system
def dynamics(t, states, control):
    x, y, vx, vy = states[:4]
    lambda_x, lambda_y, lambda_vx, lambda_vy = states[4:]
    ax, ay = control
    # State dynamics
    dxdt = vx
    dydt = vy
    dvxdt = ax
    dvydt = ay
    # Costate dynamics
    dlambda_xdt = 0  # since H does not explicitly depend on x
    dlambda_ydt = 0  # since H does not explicitly depend on y
    dlambda_vxdt = -lambda_x
    dlambda_vydt = -lambda_y
    return [dxdt, dydt, dvxdt, dvydt, dlambda_xdt, dlambda_ydt, dlambda_vxdt, dlambda_vydt]

# The running cost is simply time, so L = 1 and does not affect the Hamiltonian minimization directly.
# Thus, we only need to find the control that minimizes the inner product of lambda and control vector.

# Function to determine the optimal control given a costate
def optimal_control(lambda_vx, lambda_vy):
    # If costate is zero, control is undefined but we can set it to zero
    if lambda_vx == 0 and lambda_vy == 0:
        return (0, 0)
    
    # Direction of the maximum control is opposite to the direction of lambda (bang-bang control)
    angle = np.arctan2(-lambda_vy, -lambda_vx)
    ax = a_max * np.cos(angle)
    ay = a_max * np.sin(angle)
    return (ax, ay)

# The shooting method to solve the optimal control problem
def shooting_method(initial_state, initial_lambda, final_state):
    def boundary_conditions(lambda_guess):
        # Simulate the system with guessed costates
        lambda_initial = lambda_guess
        states_initial = np.concatenate((initial_state, lambda_initial))

        # Integrate the system of equations
        result = solve_ivp(lambda t, y: dynamics(t, y, optimal_control(y[6], y[7])),
                   [0, np.inf],  # we don't know the final time, so we integrate to "infinity" (large number)
                   states_initial, 
                   method='Radau', # Using a solver that's suitable for stiff ODEs
                   atol=1e-6, rtol=1e-3, # Set tolerances
                   events=lambda t, y: event_reach_final(t, y, final_state),
                   dense_output=True)
        
        # Extract the final state from the result
        final_t = result.t_events[0][0]
        final_states = result.sol(final_t)
        
        # Compute the error between the final states and the desired final states (excluding velocities)
        error = final_states[:2] - final_state[:2]
        return error
    
    def event_reach_final(t, y, final_state):
        # Stop integration when the particle reaches the vicinity of the final position
        return np.linalg.norm(y[:2] - final_state[:2]) - 0.01  # 0.01 is a tolerance for reaching the final state
    
    event_reach_final.terminal = True  # Stop the integration when this event is triggered

    # Run the optimizer to adjust the costates to meet the boundary conditions
    opt_result = minimize(boundary_conditions, initial_lambda, method='Nelder-Mead')
    
    if not opt_result.success:
        raise ValueError("The optimization did not converge to a solution. Try different initial guesses or methods.")
    
    return opt_result.x

# Example usage: Solve the problem for a given initial velocity
initial_velocity = 1.0  # Change this to vary the initial speed
initial_state = np.array([0, 0, initial_velocity, 0])  # Initial position and velocity
final_state = np.array([1, 1, 0, initial_velocity])    # Desired final position and final velocity
initial_lambda = np.random.rand(4) * 2 - 1             # Random initial guess for lambda

# Use the shooting method to find the optimal initial costates
optimal_lambda = shooting_method(initial_state, initial_lambda, final_state)
print(f"Optimal initial costates: {optimal_lambda}")

# With the optimal initial costates, integrate the system to get the full trajectory
optimal_states_initial = np.concatenate((initial_state, optimal_lambda))
optimal_result = solve_ivp(lambda t, y: dynamics(t, y, optimal_control(y[6], y[7])),
                           [0, np.inf],  # we don't know the final time, so we integrate to "infinity"
                           optimal_states_initial, 
                           events=lambda t, y: event_reach_final(t, y, final_state),
                           dense_output=True)

# Extract and plot the trajectory
optimal_trajectory = optimal_result.sol(optimal_result.t_events[0])
print("Optimal trajectory:")
print(optimal_trajectory)


/opt/anaconda3/lib/python3.11/site-packages/scipy/integrate/_ivp/rk.py:63: RuntimeWarning: overflow encountered in multiply
  dy = np.dot(K[:s].T, a[:s]) * h
/opt/anaconda3/lib/python3.11/site-packages/scipy/integrate/_ivp/rk.py:66: RuntimeWarning: overflow encountered in multiply
  y_new = y + h * np.dot(K[:-1].T, B)
/opt/anaconda3/lib/python3.11/site-packages/scipy/integrate/_ivp/rk.py:106: RuntimeWarning: overflow encountered in multiply
  return np.dot(K.T, self.E) * h
/opt/anaconda3/lib/python3.11/site-packages/scipy/integrate/_ivp/rk.py:109: RuntimeWarning: invalid value encountered in divide
  return norm(self._estimate_error(K, h) / scale)


KeyboardInterrupt: 